In [1]:
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*
import ro.jf.funds.reporting.api.model.*
import java.math.BigDecimal

val username = "Johann-61.0"

val monthlyReportFrom = YearMonthTO(2020, 1)
val monthlyReportTo = YearMonthTO(2024, 1)
val monthlyReportForecastUntil = YearMonthTO(2024, 7)

val yearlyReportFrom = 2019
val yearlyReportTo = 2023
val yearlyReportForecastUntil = 2025

val workFundName = "Work Income"
val workReportViewName = "Work income report"
val WORK_REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/work-report-data-configuration.yaml"

val savingsFundName = "Savings"
val savingsReportViewName = "Savings report"
val SAVINGS_REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/savings-report-data-configuration.yaml"

val otherIncomeFundName = "Other Income"
val otherIncomeReportViewName = "Other income report"
val OTHER_INCOME_REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/other-income-report-data-configuration.yaml"

val client = FundsClient()

In [2]:
val user = client.ensureUserExists(username)
user

UserTO(id=d2684936-3aec-411c-a402-5f13a03dff05, username=Johann-61.0)

In [3]:
val workReportView = client.createReportView(user, workReportViewName, workFundName, client.fromYaml(File(WORK_REPORT_DATA_CONFIGURATION_YAML_FILE), "dataConfiguration"))
workReportView

ReportViewTO(id=60dd62d0-d1f2-48ea-af55-8cf6517a96e2, name=Work income report, fundId=b39005d7-8ec4-4262-acd3-5e045a958580, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), groups=[ReportGroupTO(name=income, filter=RecordFilterTO(labels=[income])), ReportGroupTO(name=taxes, filter=RecordFilterTO(labels=[work_taxes]))], reports=ReportsConfigurationTO(net=NetReportConfigurationTO(enabled=false, filter=null), valueReport=ValueReportConfigurationTO(enabled=true, filter=null), groupedNet=GenericReportConfigurationTO(enabled=true), groupedBudget=GroupedBudgetReportConfigurationTO(enabled=false, distributions=[]), performance=GenericReportConfigurationTO(enabled=false), instrumentPerformance=GenericReportConfigurationTO(enabled=false), interestRate=GenericReportConfigurationTO(enabled=false), instrumentInterestRate=GenericReportConfigurationTO(enabled=false)), forecast=ForecastConfigurationTO(inputBuckets=1)))

In [4]:
data class GroupedNetToValueReport(
    val groupedNet: ByGroupTO<GroupNetReportTO>,
    val value: ValueReportTO
)

val monthlyWorkGroupedNetReportData = client.getReportGroupedNetData(user, workReportViewName, ReportDataIntervalTO.Monthly(monthlyReportFrom, monthlyReportTo, monthlyReportForecastUntil))
val monthlyWorkValueReportData = client.getReportValueData(user, workReportViewName, ReportDataIntervalTO.Monthly(monthlyReportFrom, monthlyReportTo, monthlyReportForecastUntil))

val monthlyWorkReportData: ReportDataTO<GroupedNetToValueReport> = monthlyWorkGroupedNetReportData.merge(monthlyWorkValueReportData, ::GroupedNetToValueReport)

In [ ]:
val yearlyWorkGroupedNetReportData = client.getReportGroupedNetData(user, workReportViewName, ReportDataIntervalTO.Yearly(yearlyReportFrom, yearlyReportTo, yearlyReportForecastUntil))
val yearlyWorkValueReportData = client.getReportValueData(user, workReportViewName, ReportDataIntervalTO.Yearly(yearlyReportFrom, yearlyReportTo, yearlyReportForecastUntil))

val yearlyWorkReportData = yearlyWorkGroupedNetReportData.merge(yearlyWorkValueReportData, ::GroupedNetToValueReport)

In [ ]:
client.plotReportData(
    title = "Yearly work income (RON)",
    reportData = yearlyWorkReportData,
    plottedLines = mapOf(
        Color.GREEN to { item -> item.groupedNet["income"].net },
        Color.YELLOW to { item -> item.groupedNet.groups.sumOf { it.net } },
        Color.RED to { item -> item.groupedNet["taxes"].net }
    ),
    plottedAreas = mapOf(
        Color.ORANGE to { item -> item.value.end }
    )
)


In [ ]:
client.plotReportData(
    title = "Monthly work income (RON)",
    reportData = monthlyWorkReportData,
    plottedLines = mapOf(
        Color.GREEN to { item -> item.groupedNet["income"].net },
        Color.YELLOW to { item -> item.groupedNet.groups.sumOf { it.net } },
        Color.RED to { item -> item.groupedNet["taxes"].net }
    ),
    plottedAreas = mapOf(
        Color.ORANGE to { item -> item.value.end }
    )
)

In [ ]:
val savingsReportView = client.createReportView(user, savingsReportViewName, savingsFundName, client.fromYaml(File(SAVINGS_REPORT_DATA_CONFIGURATION_YAML_FILE)))

In [ ]:
val monthlySavingsGroupedNetReportData = client.getReportGroupedNetData(user, savingsReportViewName, ReportDataIntervalTO.Monthly(monthlyReportFrom, monthlyReportTo, monthlyReportForecastUntil))


In [ ]:
val otherIncomeReportView = client.createReportView(user, otherIncomeReportViewName, otherIncomeFundName, client.fromYaml(File(OTHER_INCOME_REPORT_DATA_CONFIGURATION_YAML_FILE)))

In [ ]:
val monthlyOtherIncomeGroupedNetReportData = client.getReportGroupedNetData(user, otherIncomeReportViewName, ReportDataIntervalTO.Monthly(monthlyReportFrom, monthlyReportTo, monthlyReportForecastUntil))

In [ ]:
client.plotReportData(
    title = "Monthly savings income (RON)",
    reportData = monthlySavingsGroupedNetReportData,
    plottedLines = mapOf(
        Color.YELLOW to { item -> item["profit"].net },
    ),
)

In [ ]:
client.plotReportData(
    title = "Monthly other income (RON)",
    reportData = monthlyOtherIncomeGroupedNetReportData,
    plottedLines = mapOf(
        Color.YELLOW to { item -> item["income"].net },
    ),
)